In [1]:
import os
os.environ["OPENAI_API_KEY"] = "NA" # This satisfies the validator
os.environ["OTEL_SDK_DISABLED"] = "true"


# install the necessary libraries 
# !pip install crewai crewai_tools langchain_community
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
llm = "ollama/llama3.2"


In [3]:
import os
import shutil
from pathlib import Path
from crewai.tools import BaseTool
from crewai_tools import RagTool
from crewai_tools.tools.rag import RagToolConfig, ProviderSpec
from crewai.rag.embeddings.providers.ollama.types import OllamaProviderSpec
from crewai.utilities.paths import db_storage_path
from pydantic import BaseModel, Field

# Path to the knowledge base file (read and loaded into RagTool)
KNOWLEDGE_FILE = "./knowledge/deeplearning_batch.txt"

# Clear the default RAG Chroma storage so we get a fresh DB with Ollama (avoids
# "embedding function conflict: new: ollama vs persisted: openai"). Do this *before* creating RagTool.
_default_rag_dir = Path(db_storage_path())
if _default_rag_dir.exists():
    shutil.rmtree(_default_rag_dir)
_default_rag_dir.mkdir(parents=True, exist_ok=True)

embedding_model: OllamaProviderSpec = {
    "provider": "ollama",
    "config": {
        "model_name": "llama3.2",
        "url": "http://localhost:11434/api/embeddings"
    }
}

config: RagToolConfig = {
    "embedding_model": embedding_model,
}

# 1. Create RagTool with config (Chroma uses default path, now empty → Ollama)
_base_rag_tool = RagTool(config=config, collection_name="rag_deeplearning_batch_ollama")

# 2. Read the file and add it into RagTool (loads and indexes the knowledge base)
_base_rag_tool.add(data_type="file", path=KNOWLEDGE_FILE)

# 3. Create wrapper tool with proper schema (makes similarity_threshold and limit optional)
class RagToolInput(BaseModel):
    """Input schema for RAG tool - only query is required."""
    query: str = Field(..., description="The question or query to search the knowledge base")
    similarity_threshold: float | None = Field(None, description="Optional similarity threshold (defaults to tool's default)")
    limit: int | None = Field(None, description="Optional limit on number of results (defaults to tool's default)")

class WrappedRagTool(BaseTool):
    """Wrapper around RagTool with explicit optional parameters."""
    name: str = "Knowledge base"
    description: str = "A knowledge base that can be used to answer questions. Call with only the 'query' parameter."
    args_schema: type[BaseModel] = RagToolInput
    
    def _run(self, query: str, similarity_threshold: float | None = None, limit: int | None = None) -> str:
        return _base_rag_tool._run(query=query, similarity_threshold=similarity_threshold, limit=limit)

rag_tool = WrappedRagTool()

In [4]:
# Create agent with RAG tool over knowledge base (tool-calling RAG). max_iterations allows tool call + then final answer.
from crewai import Agent, Task, Crew
researcher = Agent(
    role='latest trends Analyst',
    goal='Query the RAG knowledge base tool to get content, then output the top 3 AI trends as a numbered list.',
    backstory='You use the RAG tool (Knowledge base): call it with ONLY the "query" parameter (e.g. query="What are the latest AI trends?"). Do not pass similarity_threshold or limit. Then from the tool output extract and output the top 3 trends.',
    tools=[rag_tool],
    llm=llm,
    verbose=True,
    max_iterations=3,  # enough for: call tool -> get result -> output 3 trends
)

In [5]:
# Task: pure tool-calling RAG — no context injection. Agent must query the RAG tool, then output 3 trends.
research = Task(
    description="""1) Call your RAG tool (Knowledge base) with ONLY the 'query' parameter. Example: query="What are the latest AI trends in The Batch?" 
   IMPORTANT: Do NOT pass similarity_threshold or limit parameters. Only pass the 'query' string.
2) From the tool output you receive, identify the 3 most important recent AI trends.
3) Your final answer must be ONLY these 3 numbered lines (nothing else):

1. [Trend name]: [1-2 sentences]
2. [Trend name]: [1-2 sentences]
3. [Trend name]: [1-2 sentences]

You must call the RAG tool first. After you get the tool result, use it to write the 3 trends above.""",
    expected_output="Exactly 3 numbered lines with trend name and 1-2 sentences each.",
    agent=researcher,
)

In [6]:
# Assemble a crew with planning enabled

crew = Crew(
    agents=[researcher],
    tasks=[research],
    verbose=True,
    planning=False,  # Enable planning feature
    llm="ollama/llama3.2",   # for planner
)

# Execute tasks
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 89c05ef1-400d-433a-b94e-220cbebb58f3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: latest trends Analyst                                                                                   │
│                                                                                                                 │
│  Task: 1) Call your RAG tool (Knowledge base) with ONLY the 'query' parameter. Example: query="What are the     │
│  latest AI trends in The Batch?"                                                                                │
│     IMPORTANT: Do NOT pass similarity_threshold or limit parameters. Only pass the 'query' string.              │
│  2) From the tool output you receive, identify the 3 most important recent AI trends.                           │
│  3) Your final answer must be ONLY these 3 numbered lines (nothing else):                                       │
│                                                                                                                 │
│  1. [Trend name]: [1-2 sentences]                                                                               │
│  2. [Trend name]: [1-2 sentences]                                                                               │
│  3. [Trend name]: [1-2 sentences]                                                                               │
│                                                                                                                 │
│  You must call the RAG tool first. After you get the tool result, use it to write the 3 trends above.           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: latest trends Analyst                                                                                   │
│                                                                                                                 │
│  Thought: Action: Knowledge base                                                                                │
│                                                                                                                 │
│  Using Tool: Knowledge base                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "What are the latest AI trends?"                                                                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "What are the latest AI trends?"                                                                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: latest trends Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Explainable AI (XAI): XAI is a technique used to provide insights into machine learning models'             │
│  decision-making processes. It involves visualizing and interpreting the output of machine learning algorithms  │
│  to understand how they arrive at their predictions.                                                            │
│  2. Edge AI: Edge AI refers to the integration of artificial intelligence capabilities at the edge of a         │
│  network, such as on smartphones or smart home devices, rather than in the cloud. This approach reduces         │
│  latency and enables real-time processing of data.                                                              │
│  3. Transfer Learning: Transfer learning is a machine learning technique that allows models trained on one      │
│  task to be applied to another related task with minimal retraining required.                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9c39cad9-58ea-450a-9b77-12705a6333bd                                                                     │
│  Agent: latest trends Analyst                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 89c05ef1-400d-433a-b94e-220cbebb58f3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. Explainable AI (XAI): XAI is a technique used to provide insights into machine learning       │
│  models' decision-making processes. It involves visualizing and interpreting the output of machine learning     │
│  algorithms to understand how they arrive at their predictions.                                                 │
│  2. Edge AI: Edge AI refers to the integration of artificial intelligence capabilities at the edge of a         │
│  network, such as on smartphones or smart home devices, rather than in the cloud. This approach reduces         │
│  latency and enables real-time processing of data.                                                              │
│  3. Transfer Learning: Transfer learning is a machine learning technique that allows models trained on one      │
│  task to be applied to another related task with minimal retraining required.                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯